Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

original_train_labels = train_labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

http://rishy.github.io/ml/2015/04/28/l1-vs-l2-loss/

In [5]:
# Logistic regression.

from sklearn.linear_model import LogisticRegression

regressor = LogisticRegression(penalty='l2')
regressor.fit(train_dataset[:10000,:], original_train_labels[:10000])
prediction = regressor.predict(test_dataset)
print("Accuracy: ", accuracy((np.arange(num_labels) == prediction[:,None]).astype(np.float32), test_labels))

Accuracy:  86.27


In [6]:
# Neural network.

train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )

    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
def run_gd_training(num_steps):
    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized')
        for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
            _, l, predictions = session.run([optimizer, loss, train_prediction])
            if (step % 100 == 0):
                print('Loss at step {}: {}'.format(step, l))
                print('Training accuracy: {:.1f}'.format(accuracy(predictions, 
                                                             train_labels[:train_subset, :])))
                # Calling .eval() on valid_prediction is basically like calling run(), but
                # just to get that one numpy array. Note that it recomputes all its graph
                # dependencies.

                # You don't have to do .eval above because we already ran the session for the
                # train_prediction
                print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                               valid_labels)))
        print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 

In [8]:
%%time
run_gd_training(801)

Initialized
Loss at step 0: 46.4647941589
Training accuracy: 7.3
Validation accuracy: 11.2
Loss at step 100: 11.7526607513
Training accuracy: 74.9
Validation accuracy: 73.8
Loss at step 200: 4.45667219162
Training accuracy: 79.0
Validation accuracy: 77.6
Loss at step 300: 1.96321082115
Training accuracy: 82.5
Validation accuracy: 80.5
Loss at step 400: 1.11597776413
Training accuracy: 84.1
Validation accuracy: 82.1
Loss at step 500: 0.825283825397
Training accuracy: 84.6
Validation accuracy: 82.8
Loss at step 600: 0.72435939312
Training accuracy: 84.8
Validation accuracy: 83.0
Loss at step 700: 0.688986182213
Training accuracy: 85.0
Validation accuracy: 83.0
Loss at step 800: 0.676491558552
Training accuracy: 84.9
Validation accuracy: 82.9
Test accuracy: 88.9
CPU times: user 52.3 s, sys: 2.65 s, total: 54.9 s
Wall time: 55.8 s


In [9]:
# Neural network with a hidden layer.

num_nodes = 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
  
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
    # Training computation.
    logits = tf.matmul(hidden1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    l2 = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss += 1e-5 * l2 
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [10]:
def run_sgd_training(train_dataset, train_labels, num_steps, offset=None):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            if offset is None:
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [11]:
%%time
run_sgd_training(train_dataset, train_labels, 3001)

Initialized
Minibatch loss at step 0 : 397.72
Minibatch accuracy: 8.6%
Validation accuracy: 23.4%
Minibatch loss at step 500 : 3.12968
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 1000 : 3.11409
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 1500 : 3.09856
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 2000 : 3.0831
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 2500 : 3.06772
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Minibatch loss at step 3000 : 3.05243
Minibatch accuracy: 100.0%
Validation accuracy: 64.5%
Test accuracy: 69.6%
CPU times: user 2min 19s, sys: 4.98 s, total: 2min 24s
Wall time: 2min 25s


In [12]:
num_nodes= 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    relu_layer= tf.nn.relu(logits1)
    logits2 = tf.matmul(relu_layer, weights2) + biases2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
    regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training data.
    train_prediction = tf.nn.softmax(logits2)
    
    # Predictions for the validation data.
    logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    relu_layer= tf.nn.relu(logits1)
    logits2 = tf.matmul(relu_layer, weights2) + biases2
    
    valid_prediction = tf.nn.softmax(logits2)
    
    # Predictions for the test data.
    logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
    relu_layer= tf.nn.relu(logits1)
    logits2 = tf.matmul(relu_layer, weights2) + biases2
    
    test_prediction =  tf.nn.softmax(logits2)

In [13]:
%%time
run_sgd_training(train_dataset, train_labels, 3001)

Initialized
Minibatch loss at step 0 : 3512.49
Minibatch accuracy: 9.4%
Validation accuracy: 32.3%
Minibatch loss at step 500 : 20.968
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 1000 : 0.366071
Minibatch accuracy: 100.0%
Validation accuracy: 73.9%
Minibatch loss at step 1500 : 0.209852
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 2000 : 0.200672
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 2500 : 0.196143
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 3000 : 0.193224
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Test accuracy: 79.7%
CPU times: user 1min 53s, sys: 5.99 s, total: 1min 59s
Wall time: 2min


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
run_sgd_training(train_dataset, train_labels, 3001, np.random.choice(np.arange(10)))

Initialized
Minibatch loss at step 0 : 3446.93
Minibatch accuracy: 10.2%
Validation accuracy: 30.5%
Minibatch loss at step 500 : 20.9251
Minibatch accuracy: 100.0%
Validation accuracy: 70.9%
Minibatch loss at step 1000 : 0.361159
Minibatch accuracy: 100.0%
Validation accuracy: 73.9%
Minibatch loss at step 1500 : 0.205413
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Minibatch loss at step 2000 : 0.196687
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 2500 : 0.192608
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 3000 : 0.190027
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Test accuracy: 79.9%


In [15]:
run_sgd_training(train_dataset[:500, :], train_labels[:500], 3001)

Initialized
Minibatch loss at step 0 : 3544.44
Minibatch accuracy: 7.0%
Validation accuracy: 29.7%
Minibatch loss at step 500 : 20.9981
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 1000 : 0.367163
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1500 : 0.209947
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
Minibatch loss at step 2000 : 0.200584
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Minibatch loss at step 2500 : 0.195895
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 3000 : 0.192872
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Test accuracy: 79.7%


In [16]:
run_sgd_training(train_dataset[:129, :], train_labels[:129], 3001)

Initialized
Minibatch loss at step 0 : 3485.13
Minibatch accuracy: 12.5%
Validation accuracy: 24.4%
Minibatch loss at step 500 : 21.0295
Minibatch accuracy: 100.0%
Validation accuracy: 72.6%
Minibatch loss at step 1000 : 0.368165
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 1500 : 0.210133
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2000 : 0.20032
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
Minibatch loss at step 2500 : 0.195433
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
Minibatch loss at step 3000 : 0.192339
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Test accuracy: 79.4%


Training accuracy is higher than testing accuracy => overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [17]:
num_nodes= 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    relu_layer= tf.nn.relu(logits1)

    keep_prob = tf.placeholder("float")
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    
    logits2 = tf.matmul(relu_layer_dropout, weights2) + biases2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
    regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training data.
    train_prediction = tf.nn.softmax(logits2)
    
    # Predictions for the validation data.
    logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    relu_layer= tf.nn.relu(logits1)
    logits2 = tf.matmul(relu_layer, weights2) + biases2
    
    valid_prediction = tf.nn.softmax(logits2)
    
    # Predictions for the test data.
    logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
    relu_layer= tf.nn.relu(logits1)
    logits2 = tf.matmul(relu_layer, weights2) + biases2
    
    test_prediction =  tf.nn.softmax(logits2)

In [18]:
def run_sgd_training_tuned(train_dataset, train_labels, num_steps, offset=None, confidence=0.5):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            if offset is None:
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : confidence}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [19]:
run_sgd_training_tuned(train_dataset, train_labels, 3001)

Initialized
Minibatch loss at step 0 : 3622.21
Minibatch accuracy: 7.0%
Validation accuracy: 28.6%
Minibatch loss at step 500 : 21.042
Minibatch accuracy: 100.0%
Validation accuracy: 72.1%
Minibatch loss at step 1000 : 0.392735
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Minibatch loss at step 1500 : 0.228068
Minibatch accuracy: 100.0%
Validation accuracy: 74.3%
Minibatch loss at step 2000 : 0.212551
Minibatch accuracy: 100.0%
Validation accuracy: 74.6%
Minibatch loss at step 2500 : 0.217324
Minibatch accuracy: 100.0%
Validation accuracy: 74.5%
Minibatch loss at step 3000 : 0.206963
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Test accuracy: 80.5%


Extreme overfitting case:

In [20]:
run_sgd_training_tuned(train_dataset[:500, :], train_labels[:500], 3001)

Initialized
Minibatch loss at step 0 : 3621.72
Minibatch accuracy: 7.8%
Validation accuracy: 25.4%
Minibatch loss at step 500 : 21.0065
Minibatch accuracy: 100.0%
Validation accuracy: 72.0%
Minibatch loss at step 1000 : 0.381881
Minibatch accuracy: 100.0%
Validation accuracy: 74.2%
Minibatch loss at step 1500 : 0.224813
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Minibatch loss at step 2000 : 0.214306
Minibatch accuracy: 100.0%
Validation accuracy: 74.4%
Minibatch loss at step 2500 : 0.209964
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Minibatch loss at step 3000 : 0.210832
Minibatch accuracy: 100.0%
Validation accuracy: 74.6%
Test accuracy: 80.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [21]:
num_nodes_layer1 = 1024
num_nodes_layer2 = int(num_nodes_layer1 * 0.5)
num_nodes_layer3 = int(num_nodes_layer1 * 0.5**2)

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_layer1]))
    biases1 = tf.Variable(tf.zeros([num_nodes_layer1]))
  
    weights2 = tf.Variable(tf.truncated_normal([num_nodes_layer1, num_nodes_layer2]))
    biases2 = tf.Variable(tf.zeros([num_nodes_layer2]))

    weights3 = tf.Variable(tf.truncated_normal([num_nodes_layer2, num_nodes_layer3]))
    biases3 = tf.Variable(tf.zeros([num_nodes_layer3]))

    weights4 = tf.Variable(tf.truncated_normal([num_nodes_layer3, num_labels]))
    biases4 = tf.Variable(tf.zeros([num_labels]))

    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)
  
    # Training computation.
    logits = tf.matmul(hidden3, weights4) + biases4
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    l2 = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4)
    loss += 1e-5 * l2 
  
    # Optimizer.
    global_step = tf.Variable(0)
    starter_learning_rate = 0.00001
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)


In [22]:
def run_sgd_training_with_lr(train_dataset, train_labels, num_steps):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized to a few batches.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Learning rate: ", learning_rate.eval())
                print("Global step: ", global_step.eval())
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [23]:
run_sgd_training_with_lr(train_dataset, train_labels, 3001)

Initialized
Learning rate:  1e-05
Global step:  1
Minibatch loss at step 0 : 75531.8
Minibatch accuracy: 3.9%
Validation accuracy: 8.0%
Learning rate:  9.6e-06
Global step:  501
Minibatch loss at step 500 : 5217.08
Minibatch accuracy: 64.1%
Validation accuracy: 65.2%
Learning rate:  9.216e-06
Global step:  1001
Minibatch loss at step 1000 : 4643.64
Minibatch accuracy: 68.0%
Validation accuracy: 70.6%
Learning rate:  8.84736e-06
Global step:  1501
Minibatch loss at step 1500 : 3903.38
Minibatch accuracy: 75.8%
Validation accuracy: 72.3%
Learning rate:  8.49346e-06
Global step:  2001
Minibatch loss at step 2000 : 3051.69
Minibatch accuracy: 74.2%
Validation accuracy: 73.6%
Learning rate:  8.15373e-06
Global step:  2501
Minibatch loss at step 2500 : 3706.82
Minibatch accuracy: 71.9%
Validation accuracy: 74.2%
Learning rate:  7.82758e-06
Global step:  3001
Minibatch loss at step 3000 : 2099.92
Minibatch accuracy: 78.9%
Validation accuracy: 74.9%
Test accuracy: 81.3%


In [24]:
import math

batch_size = 128
beta = 0.001

num_nodes_layer1 = 1024
num_nodes_layer2 = int(num_nodes_layer1 * 0.5)
num_nodes_layer3 = int(num_nodes_layer1 * 0.5**2)
num_nodes_layer4 = int(num_nodes_layer1 * 0.5**3)
num_nodes_layer5 = int(num_nodes_layer1 * 0.5**4)

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_layer1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases1 = tf.Variable(tf.zeros([num_nodes_layer1]))

    weights2 = tf.Variable(tf.truncated_normal([num_nodes_layer1, num_nodes_layer2], stddev=math.sqrt(2.0/num_nodes_layer1)))
    biases2 = tf.Variable(tf.zeros([num_nodes_layer2]))
    
    weights3 = tf.Variable(tf.truncated_normal([num_nodes_layer2, num_nodes_layer3], stddev=math.sqrt(2.0/num_nodes_layer2)))
    biases3 = tf.Variable(tf.zeros([num_nodes_layer3]))
    
    weights4 = tf.Variable(tf.truncated_normal([num_nodes_layer3, num_nodes_layer4], stddev=math.sqrt(2.0/num_nodes_layer3)))
    biases4 = tf.Variable(tf.zeros([num_nodes_layer4]))
    
    weights5 = tf.Variable(tf.truncated_normal([num_nodes_layer4, num_nodes_layer5], stddev=math.sqrt(2.0/num_nodes_layer4)))
    biases5 = tf.Variable(tf.zeros([num_nodes_layer5]))
    
    weights6 = tf.Variable(tf.truncated_normal([num_nodes_layer5, num_labels], stddev=math.sqrt(2.0/num_nodes_layer5)))
    biases6 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    hidden_layer1 = tf.nn.relu(logits1)
    keep_prob = tf.placeholder("float")
    hidden_layer1_dropout = tf.nn.dropout(hidden_layer1, keep_prob)
    
    logits2 = tf.matmul(hidden_layer1_dropout, weights2) + biases2
    hidden_layer2 = tf.nn.relu(logits2)
    hidden_layer2_dropout = tf.nn.dropout(hidden_layer2, keep_prob)
    
    logits3 = tf.matmul(hidden_layer2_dropout, weights3) + biases3
    hidden_layer3 = tf.nn.relu(logits3)
    hidden_layer3_dropout = tf.nn.dropout(hidden_layer3, keep_prob)
    
    logits4 = tf.matmul(hidden_layer3_dropout, weights4) + biases4
    hidden_layer4 = tf.nn.relu(logits4)
    hidden_layer4_dropout = tf.nn.dropout(hidden_layer4, keep_prob)
    
    logits5 = tf.matmul(hidden_layer4_dropout, weights5) + biases5
    hidden_layer5 = tf.nn.relu(logits5)
    hidden_layer5_dropout = tf.nn.dropout(hidden_layer5, keep_prob)
    
    logits6 = tf.matmul(hidden_layer5_dropout, weights6) + biases6 
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits6, tf_train_labels))

    regularizers = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + \
                   tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4) + \
                   tf.nn.l2_loss(weights5) + tf.nn.l2_loss(weights6)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    global_step = tf.Variable(0)
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training data.
    train_prediction = tf.nn.softmax(logits6)
    
    # Predictions for validation data.
    valid_logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    valid_relu1 = tf.nn.relu(valid_logits1)
    
    valid_logits2 = tf.matmul(valid_relu1, weights2) + biases2
    valid_relu2 = tf.nn.relu(valid_logits2)
    
    valid_logits3 = tf.matmul(valid_relu2, weights3) + biases3
    valid_relu3 = tf.nn.relu(valid_logits3)
    
    valid_logits4 = tf.matmul(valid_relu3, weights4) + biases4
    valid_relu4 = tf.nn.relu(valid_logits4)
    
    valid_logits5 = tf.matmul(valid_relu4, weights5) + biases5
    valid_relu5 = tf.nn.relu(valid_logits5)
    
    valid_logits6 = tf.matmul(valid_relu5, weights6) + biases6
    
    valid_prediction = tf.nn.softmax(valid_logits6)
    
    # Predictions for test data.
    test_logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
    test_relu1 = tf.nn.relu(test_logits1)
    
    test_logits2 = tf.matmul(test_relu1, weights2) + biases2
    test_relu2 = tf.nn.relu(test_logits2)
    
    test_logits3 = tf.matmul(test_relu2, weights3) + biases3
    test_relu3 = tf.nn.relu(test_logits3)
    
    test_logits4 = tf.matmul(test_relu3, weights4) + biases4
    test_relu4 = tf.nn.relu(test_logits4)
    
    test_logits5 = tf.matmul(test_relu4, weights5) + biases5
    test_relu5 = tf.nn.relu(test_logits5)
    
    test_logits6 = tf.matmul(test_relu5, weights6) + biases6
    
    test_prediction = tf.nn.softmax(test_logits6)

In [25]:
def run_sgd_training_with_lr_tuned(train_dataset, train_labels, num_steps, confidence=0.5):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in xrange(num_steps):
            # Pick an offset within the training data, which has been randomized to a few batches.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : confidence}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Learning rate: ", learning_rate.eval())
                print("Global step: ", global_step.eval())
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [26]:
run_sgd_training_with_lr_tuned(train_dataset, train_labels, 10000)

Initialized
Learning rate:  0.5
Global step:  1
Minibatch loss at step 0 : 5.10509
Minibatch accuracy: 7.8%
Validation accuracy: 12.2%
Learning rate:  0.5
Global step:  501
Minibatch loss at step 500 : 1.75584
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
Learning rate:  0.5
Global step:  1001
Minibatch loss at step 1000 : 1.87518
Minibatch accuracy: 73.4%
Validation accuracy: 84.5%
Learning rate:  0.5
Global step:  1501
Minibatch loss at step 1500 : 1.18907
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Learning rate:  0.5
Global step:  2001
Minibatch loss at step 2000 : 1.21976
Minibatch accuracy: 78.9%
Validation accuracy: 85.4%
Learning rate:  0.5
Global step:  2501
Minibatch loss at step 2500 : 1.23374
Minibatch accuracy: 75.8%
Validation accuracy: 86.0%
Learning rate:  0.5
Global step:  3001
Minibatch loss at step 3000 : 0.831508
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
Learning rate:  0.5
Global step:  3501
Minibatch loss at step 3500 : 0.805972
Minibatc